In [1]:
import warnings

# Ignorar FutureWarning específicos de anndata
warnings.simplefilter(action='ignore', category=FutureWarning)
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import gzip
from matplotlib.pyplot import rc_context
import anndata as ad  # Importa anndata
import numpy as np

sc.set_figure_params(dpi=100)

In [2]:
# Ruta al archivo Excel
file_path = '/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/FIRMAS_SH-TCR/01-ALL_Signatures_TCR_Spec_Byst_Update241220.xlsx'

# Cargar el archivo Excel
excel_data = pd.ExcelFile(file_path)

# Leer la primera hoja del archivo
first_sheet = excel_data.parse(excel_data.sheet_names[0])

# Lista de firmas a INCLUIR
firmas_incluir = [
    "Firma_Oliveira_UP", "Firma_Oliveira_DOWN", "Lowery_Patente_UP (CD8)",
    "Lowery_Patente_DOWN (CD8)", "Lowery_Patente-2_UP (CD8)",
    "Lowery_Patente-2_DOWN (CD8)", "Lowery_ALL", "FirmaCima_UP",
    "FirmaCima_DOWN", "FirmaCimaPlus_UP", "FirmaCimaPlus_DOWN",
    "Petremand_UP", "Petremand_DOWN"
]

# Filtrar las firmas transcriptómicas seleccionadas
firmas_selected = {col: first_sheet[col].dropna().tolist() for col in first_sheet.columns if col in firmas_incluir}

# Mostrar las primeras claves y valores para verificar
print({key: firmas_selected[key][:5] for key in list(firmas_selected.keys())[:5]})

{'FirmaCima_UP': ['ARL6IP1', 'CCL3', 'CTSB', 'CXCR6', 'LAG3'], 'FirmaCima_DOWN': ['ARL4C', 'CD69', 'DAPL1', 'EMB', 'FOS'], 'FirmaCimaPlus_UP': ['CD39', 'CXCL13', 'ARL6IP1', 'CCL3', 'CTSB'], 'FirmaCimaPlus_DOWN': ['ARL4C', 'CD69', 'DAPL1', 'EMB', 'FOS'], 'Firma_Oliveira_UP': ['CXCL13', 'ZBED2', 'LAG3', 'CTLA4', 'TOX']}


In [60]:
adata = ad.read_h5ad("/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/Results/04_datos_concatenados/adata_concatenados_10_pt.h5ad")
adata

AnnData object with n_obs × n_vars = 53542 × 23209
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'prediction', 'doublet', 'doublet_filtered', 'Sample', 'n_genes', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo'
    layers: 'counts_soupx_crude', 'original_counts'

In [14]:
# Convertir las firmas seleccionadas en una lista de genes únicos
firmas_genes = [gene for genes in firmas_selected.values() for gene in genes]

# Verificar si los genes están en adata.var.index
missing_genes = [gene for gene in firmas_genes if gene not in adata.var.index]

# Crear un DataFrame para los genes faltantes
missing_genes_df = pd.DataFrame(missing_genes, columns=["Missing_Genes"])

# Guardar el DataFrame en un archivo CSV para su revisión
missing_genes_df.to_csv("missing_genes.csv", index=False)


In [15]:
# Convertir los genes faltantes a una lista formateada
genes = missing_genes_df["Missing_Genes"].tolist()

# Mostrar la lista en formato Python
print(f"genes = {genes}")

genes = ['GM2682', 'GRAMD3', 'CD39', 'GM2682', 'GRAMD3', 'AC243829.4', 'KIAA1324', 'RARRES3', 'AFAP1IL2', 'HMOX1+', ' PDLIM4', 'RARRES3', 'AC004687.1', 'AC022706.1', 'AC243960.1', 'CARS', 'PLA2G16', 'FAM49A', 'AC243829.4', 'AC243960.1', 'AC016747.1', 'WARS', 'UpP1', 'AC004585.1', 'AC034238.1', 'AC013264.1']


In [7]:
from mygene import MyGeneInfo

In [51]:
#"GENE" in adata.var_names

True

In [48]:
# Diccionario de mapeo según tus observaciones
gene_mapping = {
    "GM2682": "GM2682",  # no existe, se queda igual
    "GRAMD3": "GRAMD2B",
    "CD39": "ENTPD1",
    "AC243829.4": "AC243829.4",  # ambiguo, se queda igual
    "KIAA1324": "ELAPOR1",
    "RARRES3": "PLAAAT4",
    "AFAP1IL2": "AFAP1L2",
    "HMOX1+": "HMOX1",
    "PDLIM4": "PDLIM4",  # se queda igual
    "AC004687.1": "AC004687.1",  # ambiguo, se queda igual
    "AC022706.1": "AC022706.1",  # no existe, se queda igual
    "AC243960.1": "AC243960.1",  # ambiguo, se queda igual
    "CARS": "CARS",  # no está, se queda igual
    "PLA2G16": "PLA2G16",  # no está, se queda igual
    "FAM49A": "FAM49A",  # no está, se queda igual
    "AC016747.1": "AC016747.1",  # no está, se queda igual
    "WARS": "WARS1",
    "UpP1": "UPP1",
    "AC004585.1": "AC004585.1",  # no está, se queda igual
    "AC034238.1": "AC034238.1",  # no está, se queda igual
    "AC013264.1": "AC013264.1",  # no está, se queda igual
}

In [49]:
# Modificar firmas_selected usando el mapeo
firmas_selected = {
    key: [gene_mapping.get(gene, gene) for gene in genes]
    for key, genes in firmas_selected.items()
}


## Aplicamos las firmas transcriptómicas

In [52]:
import pandas as pd
import decoupler as dc
import scanpy as sc
# Actualiza las importaciones para futuras versiones
from anndata.io import read_csv, read_excel, read_hdf, read_loom, read_mtx, read_text, read_umi_tools

## Corremos las firmas transcriptomicas

In [61]:
# Normalizar los datos en la capa "counts_soupx_crude"
sc.pp.normalize_total(adata, target_sum=1e4)
# Log-transformación de la capa normalizada
sc.pp.log1p(adata)

In [54]:
# Convertir firmas transcriptómicas en un formato similar a reactome
firmas_transcriptomicas = pd.DataFrame(
    [(firma, gene) for firma, genes in firmas_selected.items() for gene in genes],
    columns=["geneset", "genesymbol"]
)

In [62]:
aucell_scores = dc.run_aucell(
    adata,  # Pasa directamente el objeto AnnData
    firmas_transcriptomicas,  # El DataFrame convertido
    source="geneset",  # Columna con los nombres de las firmas
    target="genesymbol",  # Columna con los genes
    use_raw=False,  # Usar la capa activa en AnnData
    n_up=None,  # Usa por defecto el top 5% de genes
    min_n=5,  # Mínimo de genes requeridos en el conjunto
    seed=42,  # Para reproducibilidad
    verbose=True
)

1 features of mat are empty, they will be removed.
Running aucell on mat with 53542 samples and 23208 targets for 13 sources.


  0%|          | 0/53542 [00:00<?, ?it/s]

In [63]:
adata.obsm['aucell_estimate'].head()

source,FirmaCimaPlus_DOWN,FirmaCimaPlus_UP,FirmaCima_DOWN,FirmaCima_UP,Firma_Oliveira_DOWN,Firma_Oliveira_UP,Lowery_ALL,Lowery_Patente-2_DOWN (CD8),Lowery_Patente-2_UP (CD8),Lowery_Patente_DOWN (CD8),Lowery_Patente_UP (CD8),Petremand_DOWN,Petremand_UP
AAACCTGAGAAGAAGC-1_PT14,0.248672,0.060685,0.248672,0.070738,0.161193,0.078694,0.155210,0.692601,0.222707,0.692601,0.246246,0.164356,0.061599
AAACCTGAGGACAGAA-1_PT14,0.136949,0.138832,0.136949,0.078389,0.156834,0.101168,0.120862,0.406231,0.239189,0.406231,0.258520,0.090081,0.081588
AAACCTGAGTGGAGTC-1_PT14,0.000000,0.194811,0.000000,0.227082,0.034970,0.143750,0.169152,0.180355,0.281758,0.180355,0.281260,0.040914,0.089662
AAACCTGAGTGTCTCA-1_PT14,0.025071,0.067682,0.025071,0.078894,0.089961,0.015393,0.061002,0.519256,0.073278,0.519256,0.081023,0.078299,0.041996
AAACCTGCAAACTGTC-1_PT14,0.222988,0.051892,0.222988,0.060488,0.120701,0.040170,0.086224,0.574989,0.080124,0.574989,0.088593,0.140071,0.055720


In [57]:
adata.obsm['aucell_estimate'].describe()

source,FirmaCimaPlus_DOWN,FirmaCimaPlus_UP,FirmaCima_DOWN,FirmaCima_UP,Firma_Oliveira_DOWN,Firma_Oliveira_UP,Lowery_ALL,Lowery_Patente-2_DOWN (CD8),Lowery_Patente-2_UP (CD8),Lowery_Patente_DOWN (CD8),Lowery_Patente_UP (CD8),Petremand_DOWN,Petremand_UP
count,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000,53542.000000
mean,0.199079,0.114751,0.199079,0.124414,0.113797,0.059599,0.117233,0.476243,0.158860,0.476243,0.173223,0.103615,0.073870
std,0.089287,0.084446,0.089287,0.084860,0.064228,0.046332,0.045816,0.159915,0.112460,0.159915,0.121232,0.035929,0.036578
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015395,0.000000,0.000000,0.000000,0.000000,0.001595,0.000000
25%,0.135110,0.055499,0.135110,0.063592,0.067616,0.028397,0.084660,0.364171,0.077617,0.364171,0.085166,0.077952,0.048052
50%,0.193959,0.099573,0.193959,0.113325,0.111757,0.045898,0.107533,0.486975,0.128441,0.486975,0.140645,0.102697,0.066032
75%,0.257966,0.156542,0.257966,0.173524,0.156414,0.075568,0.138983,0.595932,0.208001,0.595932,0.227556,0.127589,0.091775
max,0.584559,0.569014,0.584559,0.597661,0.417708,0.338480,0.340604,0.907486,0.655106,0.907486,0.723444,0.249817,0.304417


In [67]:
# Crear una copia del objeto adata
adata_copy = adata.copy()

# Trabaja con adata_copy en lugar del original
aucell_estimate = adata_copy.obsm['aucell_estimate']

# Crear las columnas "FINAL" con nombres correctos y definidos manualmente
aucell_estimate["FirmaCimaPlus_FINAL"] = aucell_estimate["FirmaCimaPlus_UP"] - aucell_estimate["FirmaCimaPlus_DOWN"]
aucell_estimate["FirmaCima_FINAL"] = aucell_estimate["FirmaCima_UP"] - aucell_estimate["FirmaCima_DOWN"]
aucell_estimate["Firma_Oliveira_FINAL"] = aucell_estimate["Firma_Oliveira_UP"] - aucell_estimate["Firma_Oliveira_DOWN"]
aucell_estimate["Lowery_Patente-2_FINAL(CD8)"] = aucell_estimate["Lowery_Patente-2_UP (CD8)"] - aucell_estimate["Lowery_Patente-2_DOWN (CD8)"]
aucell_estimate["Lowery_Patente_FINAL(CD8)"] = aucell_estimate["Lowery_Patente_UP (CD8)"] - aucell_estimate["Lowery_Patente_DOWN (CD8)"]
aucell_estimate["Petremand_FINAL"] = aucell_estimate["Petremand_UP"] - aucell_estimate["Petremand_DOWN"]
aucell_estimate["Lowery_ALL_FINAL"] = aucell_estimate["Lowery_ALL"]  # Copiado directamente

# Eliminar las columnas originales UP y DOWN
columns_to_remove = [
    "FirmaCimaPlus_UP", "FirmaCimaPlus_DOWN",
    "FirmaCima_UP", "FirmaCima_DOWN",
    "Firma_Oliveira_UP", "Firma_Oliveira_DOWN",
    "Lowery_Patente-2_UP (CD8)", "Lowery_Patente-2_DOWN (CD8)",
    "Lowery_Patente_UP (CD8)", "Lowery_Patente_DOWN (CD8)",
    "Petremand_UP", "Petremand_DOWN",
    "Lowery_ALL"
]
aucell_estimate = aucell_estimate.drop(columns=columns_to_remove)

# Actualizar la copia del objeto adata
adata_copy.obsm['aucell_estimate'] = aucell_estimate

# Mostrar las primeras filas para verificar el resultado
print(adata_copy.obsm['aucell_estimate'].head())

source                   FirmaCimaPlus_FINAL  FirmaCima_FINAL  \
AAACCTGAGAAGAAGC-1_PT14            -0.187988        -0.177935   
AAACCTGAGGACAGAA-1_PT14             0.001884        -0.058560   
AAACCTGAGTGGAGTC-1_PT14             0.194811         0.227082   
AAACCTGAGTGTCTCA-1_PT14             0.042611         0.053823   
AAACCTGCAAACTGTC-1_PT14            -0.171096        -0.162500   

source                   Firma_Oliveira_FINAL  Lowery_Patente-2_FINAL(CD8)  \
AAACCTGAGAAGAAGC-1_PT14             -0.082499                    -0.469893   
AAACCTGAGGACAGAA-1_PT14             -0.055667                    -0.167042   
AAACCTGAGTGGAGTC-1_PT14              0.108780                     0.101403   
AAACCTGAGTGTCTCA-1_PT14             -0.074568                    -0.445978   
AAACCTGCAAACTGTC-1_PT14             -0.080532                    -0.494865   

source                   Lowery_Patente_FINAL(CD8)  Petremand_FINAL  \
AAACCTGAGAAGAAGC-1_PT14                  -0.446354        -0.102757  

In [68]:
from scipy.stats import zscore

# Aplicar z-score a cada columna en adata_copy.obsm['aucell_estimate']
adata_copy.obsm['aucell_estimate_zscore'] = adata_copy.obsm['aucell_estimate'].apply(zscore, axis=0)

# Mostrar las primeras filas para verificar los resultados
print(adata_copy.obsm['aucell_estimate_zscore'].head())

source                   FirmaCimaPlus_FINAL  FirmaCima_FINAL  \
AAACCTGAGAAGAAGC-1_PT14            -0.752329        -0.758073   
AAACCTGAGGACAGAA-1_PT14             0.625701         0.118228   
AAACCTGAGTGGAGTC-1_PT14             2.025910         2.215051   
AAACCTGAGTGTCTCA-1_PT14             0.921286         0.943197   
AAACCTGCAAACTGTC-1_PT14            -0.629738        -0.644773   

source                   Firma_Oliveira_FINAL  Lowery_Patente-2_FINAL(CD8)  \
AAACCTGAGAAGAAGC-1_PT14             -0.287170                    -0.688035   
AAACCTGAGGACAGAA-1_PT14             -0.014908                     0.678248   
AAACCTGAGTGGAGTC-1_PT14              1.653671                     1.889315   
AAACCTGAGTGTCTCA-1_PT14             -0.206695                    -0.580143   
AAACCTGCAAACTGTC-1_PT14             -0.267204                    -0.800694   

source                   Lowery_Patente_FINAL(CD8)  Petremand_FINAL  \
AAACCTGAGAAGAAGC-1_PT14                  -0.630233        -1.188962  

In [71]:
# Guardar el metadata de AUCell en un archivo TSV
output_path = "/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/Results/04_anotacion_celltypist_ProjecTIL_firmas/Analisis_firmas_TCR_AUC_10_pts.tsv"

# Extraer la matriz de AUCell scores desde adata.obsm
aucell_scores = adata_copy.obsm['aucell_estimate_zscore']

# Guardar los scores en un archivo TSV
aucell_scores.to_csv(output_path, sep="\t")

Cargamos por ejemplo el archivo con integración de Harmony:

In [ ]:
import muon as mu

In [2]:
# 1️⃣ Cargar el archivo .h5mu
mdata = mu.read("/data/scratch/LAB/enric/Proyecto_pitagoras/Analisis_pitagoras/Results/05_datos_integrados/mudata_harmony_conTCR.h5mu")

/home/evercher/miniforge3/envs/dandelion/lib/python3.11/site-packages/anndata/utils.py:349: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(msg, category, stacklevel=stacklevel)
/home/evercher/miniforge3/envs/dandelion/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/evercher/miniforge3/envs/dandelion/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_

In [74]:
aucell_scores = adata_copy.obsm['aucell_estimate_zscore']

# Añadir 'aucell_estimate' al objeto Harmony
adata_harmony.obsm['aucell_estimate'] = aucell_scores

In [ ]:
# Seleccionar las firmas disponibles en adata_harmony.obsm['aucell_estimate']
selected_firmas = adata_harmony.obsm['aucell_estimate'].columns.tolist()  # Usar todas las firmas disponibles

# Transferir los scores relevantes desde obsm a obs
adata_harmony.obs[selected_firmas] = adata_harmony.obsm['aucell_estimate'][selected_firmas]

# Representar los scores en el UMAP
sc.pl.umap(
    adata_harmony,
    color=selected_firmas,  # Añadir todas las firmas disponibles
    frameon=False,
    ncols=4,  # Distribuir los gráficos en 4 columnas
    wspace=0.4  # Ajustar el espaciado entre los subplots
)